<a href="https://colab.research.google.com/github/jun-hyeok/AAI3006-01_Machine-Learning_2021Spring/blob/main/ML_Assignment_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML - Assignment 5
2021.05.09
박준혁

In [1]:
import timeit
import numpy as np
import pandas as pd
import pandas_profiling as pp
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


사용한 data set은 [Mice Protein Expression](https://www.kaggle.com/ruslankl/mice-protein-expression)이다.

Google Colab에서 drive mount하여 data를 load하니 local에서 실행하는 경우 아래 코드를 변경해주어야 한다.

In [2]:
default_dir = "/content/drive/MyDrive/Colab Notebooks/"
# default_dir = ""
file = "Data_Cortex_Nuclear.csv"
df = pd.read_csv(default_dir+file)
df = df.dropna()

In [3]:
df.tail()

,MouseID,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,pERK_N,pJNK_N,PKCA_N,pMEK_N,pNR1_N,pNR2A_N,pNR2B_N,pPKCAB_N,pRSK_N,AKT_N,BRAF_N,CAMKII_N,CREB_N,ELK_N,ERK_N,GSK3B_N,JNK_N,MEK_N,TRKA_N,RSK_N,APP_N,Bcatenin_N,SOD1_N,MTOR_N,P38_N,pMTOR_N,DSCR1_N,AMPKA_N,NR2B_N,...,TIAM1_N,pP70S6_N,NUMB_N,P70S6_N,pGSK3B_N,pPKCG_N,CDK5_N,S6_N,ADARB1_N,AcetylH3K9_N,RRP1_N,BAX_N,ARC_N,ERBB4_N,nNOS_N,Tau_N,GFAP_N,GluR3_N,GluR4_N,IL1B_N,P3525_N,pCASP9_N,PSD95_N,SNCA_N,Ubiquitin_N,pGSK3B_Tyr216_N,SHH_N,BAD_N,BCL2_N,pS6_N,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment,Behavior,class
1045,3525_11,0.382149,0.595792,0.399101,2.527574,4.629493,0.317810,0.213235,6.219567,0.232639,1.495507,0.436070,0.393382,0.437704,0.365605,0.868464,0.741830,1.758578,2.341503,0.537786,0.775327,0.369281,0.442810,0.224877,1.113358,2.699755,1.304943,0.289216,0.335989,0.851103,0.195057,0.522876,2.335989,0.521242,0.593342,0.550041,0.950776,0.761234,0.428922,0.702410,...,0.543301,0.733252,0.157150,0.821195,0.173361,2.758650,0.296032,0.380837,1.471086,0.203968,0.176143,0.193927,0.132591,0.179531,0.199371,0.211953,0.131019,0.179410,0.126301,0.491774,0.327486,1.491653,2.310670,0.170215,1.553472,0.917251,0.245705,0.184612,0.133317,0.132591,0.122308,0.507017,0.172393,0.200097,0.221994,1.141423,Ts65Dn,Saline,S/C,t-SC-s
1046,3525_12,0.387947,0.586510,0.431650,2.527041,4.781082,0.316903,0.222111,6.307723,0.239074,1.594492,0.445819,0.388745,0.440631,0.364997,0.885652,0.763720,1.757334,2.434444,0.558970,0.785472,0.364398,0.449212,0.239473,1.137697,2.691080,1.296747,0.288166,0.340451,0.842746,0.215127,0.526442,2.300738,0.533826,0.613450,0.560567,0.963680,0.786869,0.464179,0.714827,...,0.588106,0.770505,0.160860,0.880706,0.181523,2.767660,0.304421,0.375300,1.680082,0.200144,0.194738,0.209034,0.132869,0.165545,0.195459,0.186449,0.125420,0.166026,0.135632,0.456631,0.317756,1.438011,2.330010,0.168669,1.519222,0.900529,0.261893,0.188972,0.153772,0.132869,0.121216,0.492432,0.177319,0.207593,0.235223,1.104757,Ts65Dn,Saline,S/C,t-SC-s
1047,3525_13,0.343695,0.562547,0.361243,2.412885,3.949361,0.330158,0.225620,6.060165,0.254951,1.625219,0.462773,0.404111,0.432941,0.379794,0.832289,0.695162,1.691151,2.257959,0.546503,0.770619,0.384307,0.473302,0.233643,0.998747,2.575583,1.292053,0.299574,0.334921,0.830534,0.212083,0.501128,2.189020,0.527701,0.532715,0.505139,0.865630,0.737278,0.418150,0.690649,...,0.473302,0.729506,0.136392,0.761314,0.164676,2.798554,0.286769,0.365493,1.184789,0.198303,0.179761,0.173162,0.129793,0.155877,0.192803,0.184004,0.108108,0.164362,0.128221,0.486801,0.331081,1.450031,2.265556,0.170805,1.470930,0.957888,0.232715,0.189189,0.146135,0.129793,0.121936,0.490258,0.169076,0.182590,0.243715,1.104494,Ts65Dn,Saline,S/C,t-SC-s
1048,3525_14,0.346594,0.551467,0.390353,2.445052,4.236201,0.343610,0.229239,6.023371,0.240676,1.674043,0.419692,0.397563,0.426902,0.393585,0.860020,0.723272,1.715316,2.393337,0.604426,0.807310,0.430632,0.489806,0.244654,1.039533,2.602934,1.289160,0.307061,0.344107,0.845351,0.211586,0.485082,2.226256,0.539035,0.568125,0.517653,0.977126,0.767031,0.429140,0.709846,...,0.510691,0.772501,0.142408,0.815421,0.168057,2.820142,0.283871,0.382219,1.258222,0.194650,0.197168,0.169630,0.120850,0.172463,0.191503,0.197797,0.117860,0.174351,0.133124,0.493312,0.365539,1.467663,2.307160,0.176397,1.517388,0.947915,0.231471,0.208497,0.155311,0.120850,0.136900,0.502282,0.180016,0.199685,0.269866,1.136585,Ts65Dn,Saline,S/C,t-SC-s
1049,3525_15,0.365907,0.552861,0.386275,2.473812,4.347236,0.353783,0.232784,6.085354,0.242726,1.677740,0.443986,0.402037,0.441562,0.403492,0.877061,0.723327,1.711203,2.512609,0.581959,0.839476,0.382638,0.460718,0.249758,1.054559,2.568138,1.278128,0.299467,0.356208,0.831717,0.207565,0.510427,2.215325,0.536857,0.577352,0.548982,1.000970,0.820563,0.467992,0.765034,...,0.570563,0.822260,0.145508,0.837180,0.171397,2.899563,0.281192,0.350437,1.471304,0.184342,0.205708,0.181223,0.114941,0.166251,0.189488,0.173581,0.

df column이 가진 dtype을 확인한다.


```
df.dtypes.unique()
```



In [4]:
print("data types of columns : {}".format(df.dtypes.unique()))

data types of columns : [dtype('O') dtype('float64')]


dtype이 object인 column name을 확인한다.

```
df.columns[df.dtypes==object]
```



In [5]:
objs = df.columns[df.dtypes==object]
print("object type columns : {}".format(objs))

object type columns : Index(['MouseID', 'Genotype', 'Treatment', 'Behavior', 'class'], dtype='object')


중복을 제외한 column value의 pair를 확인한다.


```
df_objs.drop_duplicates().values
```

In [6]:
df_objs = df[objs].drop("MouseID", axis=1)
df_objs.drop_duplicates().values

array([['Control', 'Memantine', 'C/S', 'c-CS-m'],
       ['Control', 'Memantine', 'S/C', 'c-SC-m'],
       ['Control', 'Saline', 'C/S', 'c-CS-s'],
       ['Control', 'Saline', 'S/C', 'c-SC-s'],
       ['Ts65Dn', 'Memantine', 'C/S', 't-CS-m'],
       ['Ts65Dn', 'Memantine', 'S/C', 't-SC-m'],
       ['Ts65Dn', 'Saline', 'C/S', 't-CS-s'],
       ['Ts65Dn', 'Saline', 'S/C', 't-SC-s']], dtype=object)

각 column value의 combination이 `df["class"]` value와 일치하므로 `df["class"]`는 label `y`로 나머지 dtype이 decimal인 column은 feature `X`로 한다.

이후 train set과 test set으로 나눈다.

In [7]:
X = df.drop(objs, axis=1).values
# le = LabelEncoder()
# le.fit(df["class"].values)
# y = le.transform(df["class"])
y = df["class"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

MLPClassifier로 GridSearch를 진행한다.

In [8]:
parameters = {"hidden_layer_sizes":((200, 100), (200, 10), (100, 10), (100,)),
              "activation":("relu", "logistic", "tanh"),
              "batch_size":(50, 100, 150, 200),
              }
grid_search = GridSearchCV(MLPClassifier(max_iter=1000), parameters, n_jobs=-1, cv=3)
start_time = timeit.default_timer()
grid_search.fit(X_train, y_train)
print("{:0.3f} seconds".format(timeit.default_timer() - start_time))

375.745 seconds


best paramter와 best score를 확인한다.

In [9]:
print(grid_search.best_params_, grid_search.best_score_)

{'activation': 'tanh', 'batch_size': 200, 'hidden_layer_sizes': (200, 10)} 0.9977324263038548


best parameter로 구성된 best estimaor로 test set에서의 accuracy를 확인한다.

In [10]:
mlp_clf = grid_search.best_estimator_
acc = mlp_clf.score(X_test, y_test)
print(acc)

0.9819819819819819


prediction과 label을 가지고, `classification_report`를 이용해 성능을 평가한다.

In [11]:
y_pred = mlp_clf.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

      c-CS-m       1.00      1.00      1.00        10
      c-CS-s       1.00      1.00      1.00        15
      c-SC-m       1.00      0.90      0.95        20
      c-SC-s       1.00      1.00      1.00        10
      t-CS-m       1.00      1.00      1.00        15
      t-CS-s       1.00      1.00      1.00         8
      t-SC-m       0.89      1.00      0.94        16
      t-SC-s       1.00      1.00      1.00        17

    accuracy                           0.98       111
   macro avg       0.99      0.99      0.99       111
weighted avg       0.98      0.98      0.98       111

